In [441]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation,Conv1D, ZeroPadding1D, BatchNormalization, Flatten
from keras.layers import AveragePooling1D, MaxPooling1D, Dropout, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras import regularizers
from keras.optimizers import Adam
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras import regularizers
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import pandas as pd

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
np.random.seed(8)
%matplotlib inline

# COMPOSITIONS OF DATASETS LISTED BELOW

In [442]:


#Training dataset formed using 
#nexus4_vehicle
#s3_still_2dp_Features
#s3_still_bagroom_Features
#s3_vehicle_1_Features
#s3_vehicle_2_Features
######################
dataset = pd.read_csv('train1.csv')


#Test dataset formed using
#s3_still_hostel_Features
#s3_vehicle_3_bandra_Features
#s3_vehicle_4_bandra_Features
#s3_vehicle_5_bandra_Features
#########################
test_dataset = pd.read_csv('test1.csv')

#Test dataset formed using
#s3_still_hostel_Features
#s3_vehicle_7_bandra_Features
#########################
test_dataset_1 = pd.read_csv('test2.csv')

X= dataset.iloc[: ,0:-1].values
Y = dataset.iloc[:,-1:].values
X_realTest = test_dataset.iloc[:,0:-1].values
Y_realTest = test_dataset.iloc[:,-1:].values
X_realTest_1 = test_dataset_1.iloc[:,0:-1].values
Y_realTest_1 = test_dataset_1.iloc[:,-1:].values

print('Shape of Training X : ',X.shape)
print('Shape of Training Y :',Y.shape)
print('Shape of Test X :',X_realTest.shape)
print('Shape of Test_1 X :',X_realTest_1.shape)

Shape of Training X :  (1781, 10)
Shape of Training Y : (1781, 1)
Shape of Test X : (369, 10)
Shape of Test_1 X : (401, 10)


In [443]:
### Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0 , stratify=Y)

print(Y_train[Y_train==1].shape[0]/Y_train[Y_train==0].shape[0])
print(Y_test[Y_test==1].shape[0]/Y_test[Y_test==0].shape[0])
print('Train with Label=1   :'+str(Y_train[Y_train==1].shape[0]))
print('Train with Label=0  :'+str(Y_train[Y_train==0].shape[0]))
print('Dev set with Label=1  :'+str(Y_test[Y_test==1].shape[0]))
print('Dev set with Label=0  :'+str(Y_test[Y_test==0].shape[0]))
print()

#Training and dev set Shapes
print('Train set X_train.shape:',X_train.shape)
print('Train set Y_train.shape:',Y_train.shape)
print('Dev set X_test.shape:',X_test.shape)
print('Dev set Y_test.shape:',Y_test.shape)
print()

#Test set shape 1
print('Test set X_realTest.shape:',X_realTest.shape)
print('Test set Y_realTest.shape:',Y_realTest.shape)
print()

#Test set shape 2
print('Test set X_realTest_1.shape:',X_realTest_1.shape)
print('Test set Y_realTest_1.shape:',Y_realTest_1.shape)
print()




0.8094027954256671
0.8121827411167513
Train with Label=1   :637
Train with Label=0  :787
Dev set with Label=1  :160
Dev set with Label=0  :197

Train set X_train.shape: (1424, 10)
Train set Y_train.shape: (1424, 1)
Dev set X_test.shape: (357, 10)
Dev set Y_test.shape: (357, 1)

Test set X_realTest.shape: (369, 10)
Test set Y_realTest.shape: (369, 1)

Test set X_realTest_1.shape: (401, 10)
Test set Y_realTest_1.shape: (401, 1)



In [444]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
X_realTest = sc_X.transform(X_realTest)
X_realTest_1 = sc_X.transform(X_realTest_1)



X_train.resize((X_train.shape[0],10,1))
X_test.resize((X_test.shape[0],10,1))
X_realTest.resize((X_realTest.shape[0]),10,1)
X_realTest_1.resize((X_realTest_1.shape[0]),10,1)


c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [445]:
def MyModel(input_shape):
    
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
   # X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv1D(4, 2, strides = 1, name = 'conv0',padding='same')(X_input)
    X = BatchNormalization(axis = -1, name = 'bn0')(X)
    X = Activation('relu')(X)
    
      
     # MAXPOOL
    X = MaxPooling1D(pool_size=2, strides=None, padding='valid')(X)
   
    
    

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dropout(0.4, name='d1')(X)
    X = Dense(1,activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='ActivityDetection')

    return model

In [446]:
### START CODE HERE ### (1 line)

ActivityDetectionModel = MyModel((10,1))
print(ActivityDetectionModel.summary())
### END CODE HERE ###

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_46 (InputLayer)        (None, 10, 1)             0         
_________________________________________________________________
conv0 (Conv1D)               (None, 10, 4)             12        
_________________________________________________________________
bn0 (BatchNormalization)     (None, 10, 4)             16        
_________________________________________________________________
activation_56 (Activation)   (None, 10, 4)             0         
_________________________________________________________________
max_pooling1d_46 (MaxPooling (None, 5, 4)              0         
_________________________________________________________________
flatten_46 (Flatten)         (None, 20)                0         
_________________________________________________________________
d1 (Dropout)                 (None, 20)                0         
__________

In [447]:
### START CODE HERE ### (1 line)
opt = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, decay = 0.0001)
ActivityDetectionModel.compile(optimizer= opt , loss = 'binary_crossentropy' , metrics=["accuracy"])
### END CODE HERE ###

In [448]:
### START CODE HERE ### (1 line)
ActivityDetectionModel.fit(x=X_train , y=Y_train, epochs = 120 , batch_size=32)
### END CODE HERE ###

Epoch 1/120
1424/1424 [==============================] - 3s 2ms/step - loss: 0.7019 - acc: 0.4775
Epoch 2/120
1424/1424 [==============================] - 0s 106us/step - loss: 0.6711 - acc: 0.5351
Epoch 3/120
1424/1424 [==============================] - 0s 104us/step - loss: 0.6274 - acc: 0.5737
Epoch 4/120
1424/1424 [==============================] - 0s 104us/step - loss: 0.6059 - acc: 0.6011
Epoch 5/120
1424/1424 [==============================] - 0s 98us/step - loss: 0.5976 - acc: 0.6032
Epoch 6/120
1424/1424 [==============================] - 0s 105us/step - loss: 0.5819 - acc: 0.6117
Epoch 7/120
1424/1424 [==============================] - 0s 134us/step - loss: 0.5421 - acc: 0.6152
Epoch 8/120
1424/1424 [==============================] - 0s 169us/step - loss: 0.5294 - acc: 0.6201
Epoch 9/120
1424/1424 [==============================] - 0s 143us/step - loss: 0.5133 - acc: 0.6678
Epoch 10/120
1424/1424 [==============================] - 0s 138us/step - loss: 0.4973 - acc: 0.7830
Ep

1424/1424 [==============================] - 0s 101us/step - loss: 0.1247 - acc: 0.9628
Epoch 83/120
1424/1424 [==============================] - 0s 94us/step - loss: 0.0998 - acc: 0.9705
Epoch 84/120
1424/1424 [==============================] - 0s 94us/step - loss: 0.1153 - acc: 0.9586
Epoch 85/120
1424/1424 [==============================] - 0s 93us/step - loss: 0.1049 - acc: 0.9705
Epoch 86/120
1424/1424 [==============================] - 0s 112us/step - loss: 0.0986 - acc: 0.9733
Epoch 87/120
1424/1424 [==============================] - 0s 108us/step - loss: 0.1105 - acc: 0.9670
Epoch 88/120
1424/1424 [==============================] - 0s 94us/step - loss: 0.1290 - acc: 0.9607
Epoch 89/120
1424/1424 [==============================] - 0s 96us/step - loss: 0.1156 - acc: 0.9614
Epoch 90/120
1424/1424 [==============================] - 0s 92us/step - loss: 0.1179 - acc: 0.9705
Epoch 91/120
1424/1424 [==============================] - 0s 95us/step - loss: 0.1160 - acc: 0.9684
Epoch 92/1

In [452]:
#Fot Dev set
preds = ActivityDetectionModel.evaluate(x=X_test , y=Y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

357/357 [==============================] - 0s 78us/step

Loss = 0.021689478422747273
Test Accuracy = 0.9971988795518207


In [453]:
#For test set
pred_real = ActivityDetectionModel.evaluate(x=X_realTest,y=Y_realTest)
print()
print ("Loss = " + str(pred_real[0]))
print ("Test Accuracy = " + str(pred_real[1]))

369/369 [==============================] - 0s 65us/step

Loss = 0.5965726639889938
Test Accuracy = 0.8888888888888888


In [454]:
#For test_1 set
pred_real_1 = ActivityDetectionModel.evaluate(x=X_realTest_1,y=Y_realTest_1)
print()
print ("Loss = " + str(pred_real_1[0]))
print ("Test Accuracy = " + str(pred_real_1[1]))

401/401 [==============================] - 0s 90us/step

Loss = 0.047396629253876145
Test Accuracy = 0.9975062344139651
